# Feedback Prize Best score 0.59

In [1]:
import pandas as pd
import os
path = ''

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv(path+"train.csv")

In [107]:
from torch.utils.data import Dataset
import numpy as np
import torch
import nltk
import re

class GrammerDataset(Dataset):
    pos_tag_vocab = ['CC',
            'WRB',
            'EX',
            'MD',
            'VBN',
            'VBD',
            'NNS',
            'RBR',
            'VBZ',
            'PRP$',
            'VB',
            'RP',
            'WP',
            'VBP',
            'JJR',
            'VBG',
            'PDT',
            'JJ',
            'JJS',
            'WDT',
            'IN',
            'DT',
            'RB',
            'NN',
            'PRP',
            'TO']

    def __init__(self, data):
        '''
        Dataset object for base model
        :param data:
        '''
        self.text,self.text_length,self.pos_tag = self.clean_text(data['full_text'])
        self.cohesion = np.array(self.feature_transformation(data['cohesion']))
        self.syntax = np.array(self.feature_transformation(data['syntax']))
        self.vocab = np.array(self.feature_transformation((data['vocabulary'])))
        self.phraseology = np.array(self.feature_transformation(data['phraseology']))
        self.grammer = np.array(self.feature_transformation(data['grammar']))
        self.conventions = np.array(self.feature_transformation(data['conventions']))
        

    def __len__(self):
        return len(self.text)
    
    def feature_transformation(self, vals):
        self.mapping = {1:0,1.5:1,2:2,2.5:3,3:4,3.5:5,4:6,4.5:7,5:8}
        newl = []
        for v in vals:
            newl.append(self.mapping[v])
        return newl
    def count_pos_tag(self,text):
        tag_dict = {}
        pos = nltk.pos_tag(text)
        tag_types = list(set([item[1] for item in pos]))
        for tag in tag_types:
            tag_dict[tag] = sum([item[1]==tag for item in pos])
        return tag_dict
    

    def clean_text(self,all_text):
        newl = []
        newlength = []
        pos_tag = []
        for text in all_text:
            text = text.replace("\n"," ").lower()
            text = text.strip()
            newl.append(text)
            newlength.append([len(t) for t in text.split()])
            pos_sentence = self.count_pos_tag(text.split())
            hot_vector_pos = []
            for po in self.pos_tag_vocab:
                if po in pos_sentence.keys():
                    hot_vector_pos.append(pos_sentence[po])
                else:
                    hot_vector_pos.append(0)
            pos_tag.append(hot_vector_pos)
            
            
        return np.array(newl),np.array(newlength),np.array(pos_tag)
    def __getitem__(self, idx):
        return self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx],self.text_length[idx],self.pos_tag[idx]

In [118]:
train=df.sample(frac=0.95,random_state=3) #random state is a seed value
test=df.drop(train.index)
train_dataset = GrammerDataset(train)
valid_dataset = GrammerDataset(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [119]:
from torchtext.data.utils import(
    get_tokenizer,
    ngrams_iterator,
)
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
ngrams = 2
def yield_tokens(data_iter, ngrams):
    for text in data_iter:
        yield iter(tokenizer(text))

myvocab = build_vocab_from_iterator(yield_tokens(train_dataset.text, ngrams), specials=["<unk>"])
myvocab.set_default_index(myvocab["<unk>"])
device = 'cuda'
def text_pipeline(x,padding=512): 
    res = myvocab(list((tokenizer(x))))
    if len(res) < padding:
        chars_to_add = int(padding-len(res))
        for i in range(chars_to_add):
            res.append(myvocab['<pad>'])
    else:
        res = res[:padding]
    return res
# self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
def collate_batch(batch):
    text_list, cohesion, syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = [], [],[],[],[],[],[],[],[]
    for data in batch:
        processed_text = text_pipeline(data[0],1024)
        text_list.append(processed_text)
        text_length.append(len(processed_text))
        cohesion.append(data[1])
        syntax.append(data[2])
        vocab.append(data[3])
        phraseology.append(data[4])
        grammer.append(data[5])
        conventions.append(data[6])
        pos_tag.append(data[8])
        
        
    cohesion = torch.tensor(cohesion, dtype=torch.int64)
    syntax = torch.tensor(syntax, dtype=torch.int64)
    vocab = torch.tensor(vocab, dtype=torch.int64)
    phraseology = torch.tensor(phraseology, dtype=torch.int64)
    grammer = torch.tensor(grammer, dtype=torch.int64)
    conventions = torch.tensor(conventions, dtype=torch.int64)
    pos_tag_vals = torch.tensor(pos_tag, dtype=torch.int64)
    text_length = torch.tensor(text_length, dtype=torch.int64)
    text_list = torch.tensor(text_list, dtype=torch.int64)
    return text_list.to(device), cohesion.to(device), syntax.to(device),vocab.to(device), phraseology.to(device),grammer.to(device), conventions.to(device),text_length.to(device),pos_tag_vals.to(device)

In [127]:
import torch.nn as nn
import torch
from transformers import DebertaTokenizer, DebertaModel
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss


class MCRMSELoss(nn.Module):
    def __init__(self, num_scored=6):
        super().__init__()
        self.rmse = nn.HuberLoss()
        self.num_scored = num_scored

    def forward(self, yhat, y):
        score = 0
        for i in range(self.num_scored):
            score += self.rmse(yhat[:, i], y[:,i]) / self.num_scored

        return score
    
    
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, bidirectional, hidden_dim, num_layers, output_dim, dropout=0.3, pad_idx=0,
                 fc_hidden2=64, fc_hidden1=256):
        super().__init__()
        self.hidden_size = fc_hidden2
        self.input_dim = input_dim
        self.bid = bidirectional
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.dropout_rate = dropout 
        self.fc_hidden = fc_hidden1

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)

        # embedded to cuda if available
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=num_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)
        

        #tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
        #model = DebertaModel.from_pretrained('microsoft/deberta-base', return_dict=True)

#         inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
#         outputs = model(**inputs,return_dict=True)

        self.num_neurons = (hidden_dim * num_layers) + 26 
        
        self.labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
        self.featurs_nn = []
        self.cohesion_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        self.syntax_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        
        self.vocabulary_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        self.phraseology_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        self.grammar_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        self.conventions_nn =  nn.Sequential(*[nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
                  nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)])
        
        self.features_nn = [self.cohesion_nn, self.syntax_nn, self.vocabulary_nn, self.phraseology_nn, self.grammar_nn,self.conventions_nn]
        
#         for lab in self.labels:
#             fc = [nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),
#                   nn.Linear(self.hidden_size, self.hidden_size), nn.ReLU(inplace=True), nn.Linear(self.hidden_size, 1)]
#             fc = nn.Sequential(*fc)
#             self.featurs_nn.append(fc)
            
        #self.featurs_nn = nn.Sequential(self.featurs_nn)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text,text_length,pos_tag):
        embedded = self.dropout(self.embedding(text))
        text_length = torch.clamp(text_length, min=1)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu(),batch_first=True,enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        res = torch.cat((hidden,pos_tag),dim=1)
#         x = hidden.unsqueeze(0).transpose(0, 1)
#         output = self.cnn1(x)
#         output = self.cnn2(output)
#         output = output.view(output.shape[0], output.shape[1] * output.shape[2])
#         res = torch.cat((output, hidden), dim=1)
        output_dict = {}
        for f_nn, feature in zip(self.features_nn,self.labels):
            newh = res.clone()
            out = f_nn(newh)
            output_dict[feature] = out
        
        last_out = torch.stack((output_dict['cohesion'],output_dict['syntax'],output_dict['vocabulary'],output_dict['phraseology'],output_dict['grammar'],output_dict['conventions']),dim=1)
        #hidden = self.fc(hidden)
        return last_out

def save_model(model, path):
    global myvocab
    cfg = model.state_dict()
    cfg['input_dim'] = model.input_dim
    cfg['embedding_dim'] = model.embedding_dim
    cfg['bidirectional'] = model.bid
    cfg['hidden_dim'] = model.hidden_dim
    cfg['num_layers'] = model.num_layers
    cfg['output_dim'] = model.output_dim
    cfg['dropout'] = model.dropout_rate
    torch.save(cfg, path)
    torch.save(myvocab,path.replace(".pth","")+'_vocab.pth')
    
def load_model(path):
    keys_to_remove = ['input_dim', 'embedding_dim', 'bidirectional', 'hidden_dim', 'num_layers', 'output_dim','dropout']
    cfg = torch.load(path)
    model = RNN_CNN(cfg['input_dim'],cfg['embedding_dim'],cfg['bidirectional'],cfg['hidden_dim'],cfg['num_layers'],cfg['output_dim'],cfg['dropout'])
    
    for key in keys_to_remove:
        cfg.pop(key)
    model.load_state_dict(cfg)
    model.eval()
    return model

def load_vocab(path):
    vocab = torch.load(path)
    return vocab
    

In [128]:
vocab_size = len(myvocab)
batch_size = 64
bidirectional = True
emb_dim = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_batch,drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_batch,drop_last=True)

model = RNN(vocab_size, emb_dim,bidirectional,hidden_dim=512,num_layers=2,output_dim=1)
model.to(device)

RNN_CNN(
  (embedding): Embedding(21500, 300, padding_idx=0)
  (rnn): LSTM(300, 512, num_layers=2, dropout=0.3, bidirectional=True)
  (cohesion_nn): Sequential(
    (0): Linear(in_features=1050, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): ReLU(inplace=True)
    (7): Linear(in_features=64, out_features=1, bias=True)
  )
  (syntax_nn): Sequential(
    (0): Linear(in_features=1050, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): ReLU(inplace=True)
    (7): Linear(in_features=64, out_features=1, bias=True)
  )
  (vocabulary_nn): Sequential(
    (0): Linear(in_features=1050, out_fea

In [116]:
def transform_labels(y_vals):
    mapping = {0: 1, 1: 1.5, 2: 2, 3: 2.5, 4: 3, 5: 3.5, 6: 4, 7: 4.5, 8: 5}
    if isinstance(y_vals, torch.Tensor):
        if len(y_vals) > 0:
            y_vals = y_vals.tolist()
            newl = []
            for i in y_vals:
                newl.append(mapping[i])
            return newl
        return mapping[y_vals.item()]
    elif isinstance(y_vals,int):
         return mapping[y_vals]
    else:
        if len(y_vals) > 0:
            y_vals = y_vals.tolist()
            newl = []
            for i in y_vals:
                newl.append(mapping[i])
            return newl
def compute_acc(predicted,gt):
    total_true = sum([predicted[i]==gt[i] for i in range(len(gt))])
    return total_true/len(gt), total_true
def compare_tensor_acc(preds, gt):
    preds = torch.softmax(preds, dim=1)
    proba, class_label = preds.max(dim=1)
    predict_total = [(transform_labels(cl), pr) for cl,pr in zip(class_label.tolist(),proba.tolist())]
    predict_only = [pt[0] for pt in predict_total]
    gra_acc,true = compute_acc(predict_only,gt.tolist())
    return predict_only,true

import math
def mcrmse_score(labels,y_size=6):
    #labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
    multiplier = (1/y_size)
    loss_sum = 0
    i = 0
    for item in labels:
        pred_vector = item[0]
        gt_vector = item[1]
        loss_sum += math.sqrt((1/len(pred_vector)) * sum([(pred_vector[i]-gt_vector[i])**2 for i in range(len(pred_vector))]))
    
    return loss_sum*multiplier

In [ ]:
loss_func = MCRMSELoss()
lr = 0.001
max_epochs = 300
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
best_loss = 9999
test_score = 0.35
save_path = 'test/best_model_rmcse_loss.pth'
for epoch in range(max_epochs):
    all_losses = [] 
    print("Epoch {}".format(epoch))
    coh_train_correct = 0
    syn_train_correct = 0
    voc_train_correct = 0
    phr_train_correct = 0
    gr_train_correct = 0
    con_train_correct = 0
    train_samples = 0
    mcrmse_s = 0

    for batch_idx, batch_data in enumerate(train_loader):
        # self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
        text,cohesion,syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = batch_data
        outputs = model(text,text_length,pos_tag)
        outputs = torch.squeeze(outputs,dim=2)
        outputs = outputs.to(torch.float32)
        gt_format = torch.stack((cohesion,syntax,vocab,phraseology,grammer,conventions),dim=1)
        gt_format = gt_format.to(torch.float32)
#         gt_format = gt_format.reshape(batch_size,6)
        #gt_format = gt_format.view(1,0)



        # loss
        loss = loss_func(outputs,gt_format)
        
        
        if batch_idx % 100 == 0:
            print("Loss: {}".format(loss.item()))

        all_losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    def avg(lst):
        return sum(lst) / len(lst)
    
#     # calculating mcrmse metric
#     mcrmse_epoch_score = mcrmse_s/len(train_loader)
#     if mcrmse_epoch_score < best_loss:
#         best_loss = mcrmse_epoch_score
#         #save_model(model,save_path)
#         print("New model saved! better average RMCMSE score ", best_loss)
    avg_loss = np.mean(all_losses)
    if avg_loss < best_loss:
        best_loss = avg_loss
        save_model(model,save_path)
        print("New model saved! better loss average ", best_loss)    
    if epoch % 10 == 0:
        print("Average loss: ", avg(all_losses))

    valid_loss = []
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(valid_loader):
            text,cohesion,syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = batch_data
            outputs = model(text,text_length,pos_tag)
            outputs = torch.squeeze(outputs,dim=2)
            outputs = outputs.to(torch.float32)
            gt_format = torch.stack((cohesion,syntax,vocab,phraseology,grammer,conventions),dim=1)
            gt_format = gt_format.to(torch.float32)



            # loss
            loss = loss_func(outputs,gt_format)
            valid_loss.append(loss.item())
    
    print("Validation loss: ", np.mean(valid_loss))
    



Epoch 0
Loss: 3.8244125843048096
New model saved! better loss average  1.003210727510781
Average loss:  1.003210727510781
Validation loss:  0.7136328419049581
Epoch 1
Loss: 0.6660494804382324
New model saved! better loss average  0.6312715390632893
Validation loss:  0.6571313937505087
Epoch 2
Loss: 0.591296374797821
New model saved! better loss average  0.596413983353253
Validation loss:  0.6502397060394287
Epoch 3
Loss: 0.6356393098831177
New model saved! better loss average  0.5865801819439592
Validation loss:  0.6268380880355835
Epoch 4
Loss: 0.7281327247619629
Validation loss:  0.6577691634496053
Epoch 5
Loss: 0.552669882774353
New model saved! better loss average  0.5551371507603546
Validation loss:  0.6820014119148254
Epoch 6
Loss: 0.5448476672172546
New model saved! better loss average  0.5364593447282396
Validation loss:  0.6414553125699362
Epoch 7
Loss: 0.4563513994216919
New model saved! better loss average  0.5238961007060676
Validation loss:  0.6794151862462362
Epoch 8
Loss

In [90]:
from torch.utils.data import Dataset
import numpy as np
import torch
class GrammerTestDataset(Dataset):
    pos_tag_vocab = ['CC',
            'WRB',
            'EX',
            'MD',
            'VBN',
            'VBD',
            'NNS',
            'RBR',
            'VBZ',
            'PRP$',
            'VB',
            'RP',
            'WP',
            'VBP',
            'JJR',
            'VBG',
            'PDT',
            'JJ',
            'JJS',
            'WDT',
            'IN',
            'DT',
            'RB',
            'NN',
            'PRP',
            'TO']
    def __init__(self, data):
        '''
        Dataset object for base model
        :param data:
        '''
        self.text,self.text_length,self.pos_tag = self.clean_text(data['full_text'])
        self.text_labels = data['text_id'].to_numpy()
        

    def __len__(self):
        return len(self.text)
    
    def feature_transformation(self, vals):
        self.mapping = {1:0,1.5:1,2:2,2.5:3,3:4,3.5:5,4:6,4.5:7,5:8}
        newl = []
        for v in vals:
            newl.append(self.mapping[v])
        return newl
    
    def count_pos_tag(self,text):
        tag_dict = {}
        pos = nltk.pos_tag(text)
        tag_types = list(set([item[1] for item in pos]))
        for tag in tag_types:
            tag_dict[tag] = sum([item[1]==tag for item in pos])
        return tag_dict

    def clean_text(self,all_text):
        newl = []
        newlength = []
        pos_tag = []
        for text in all_text:
            text = text.replace("\n"," ").lower()
            text = text.strip()
            newl.append(text)
            newlength.append([len(t) for t in text.split()])
            pos_sentence = self.count_pos_tag(text.split())
            hot_vector_pos = []
            for po in self.pos_tag_vocab:
                if po in pos_sentence.keys():
                    hot_vector_pos.append(pos_sentence[po])
                else:
                    hot_vector_pos.append(0)
            pos_tag.append(hot_vector_pos)
        return np.array(newl),np.array(newlength),np.array(pos_tag)
    def __getitem__(self, idx):
        return self.text[idx],self.text_length[idx],self.pos_tag[idx],self.text_labels[idx]

In [97]:
testdf = pd.read_csv(path+"test.csv")
testdata = GrammerTestDataset(testdf)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [98]:
# self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
def collate_test_batch(batch):
    text_list,text_length,pos_tag,text_id = [], [],[],[]
    for data in batch:
        processed_text = text_pipeline(data[0],512)
        text_list.append(processed_text)
        text_length.append(len(processed_text))
        pos_tag.append(data[2])
        text_id.append(data[3])
        
        
        
    text_length = torch.tensor(text_length, dtype=torch.int64)
    text_list = torch.tensor(text_list, dtype=torch.int64)
    pos_tag = torch.tensor(pos_tag, dtype=torch.int64)
    return text_list.to(device), text_length.to(device),pos_tag.to(device),text_id

In [99]:

def test(model,loader,return_preds=False):
    coh_correct = 0
    syntax_correct = 0
    vocab_correct = 0
    phraseology_correct = 0
    grammar_correct = 0
    conventions_correct = 0
    total_samples = 0
    
    coh_preds = []
    syntax_preds = []
    voc_preds = []
    phr_preds = []
    gra_preds = []
    conv_preds = []
    text_original = []
    
    final_results = pd.DataFrame()
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(loader):
            # self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
            text = batch_data[0]
            text_length = batch_data[1]
            pos_tag = batch_data[2]
            text_id = batch_data[3]
            outputs = model(text,text_length,pos_tag)
            
            outputs = outputs.tolist()
            for item,t in zip(outputs,text_id):
                coh_preds.append(item[0][0])
                syntax_preds.append(item[1][0])
                voc_preds.append(item[2][0])
                phr_preds.append(item[3][0])
                gra_preds.append(item[4][0])
                conv_preds.append(item[5][0])   
                text_original.append(t)
            
            labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
            
            
           
    
    
    final_results['text_id'] = text_original
    final_results['cohesion'] = coh_preds
    final_results['syntax'] = syntax_preds
    final_results['vocabulary'] = voc_preds
    final_results['phraseology'] = phr_preds
    final_results['grammar'] = gra_preds
    final_results['conventions'] = conv_preds
    
    if return_preds:
        return final_results
 
        
        
    return coh_correct,syntax_correct,vocab_correct,phraseology_correct,grammar_correct,conventions_correct
    
    
                            
        


In [100]:
testloader = DataLoader(testdata, batch_size=3, shuffle=False,collate_fn=collate_test_batch)
newmodel = load_model(save_path)
newmodel.to(device)
all_res = test(newmodel,testloader,return_preds=True)

In [101]:
all_res

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,078201045481,2.531787,2.239652,2.661514,0.016086,0.012376,0.112138
1,079FDB42E429,4.388169,4.000633,4.386132,0.008437,0.014640,0.114995
2,07A4DAFDE5B7,4.602616,4.203226,4.657094,0.007949,0.011250,0.110394
3,07C6BB6ADA38,4.361373,3.972078,4.376444,0.007702,0.012865,0.111441
4,07CE77EA56C5,5.728912,5.167780,5.509420,0.009224,0.010871,0.107097
...,...,...,...,...,...,...,...
95,0EC7D67618F4,4.151865,3.810284,4.248954,0.004842,0.011698,0.110117
96,0ED9E0B4AC30,3.856188,3.538497,3.952511,0.006845,0.015726,0.114750
97,0EE67777ABB5,3.969946,3.617338,4.018198,0.006614,0.012759,0.114321
98,0EEE49F99224,3.909279,3.573855,3.980236,0.005911,0.012232,0.112401


In [34]:
import math
def mcrmse(predicted_df,gt_df,y_size=6):
    labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
    multiplier = (1/y_size)
    loss_sum = 0
    for lab in labels:
        pred_vector = predicted_df[lab].to_list()
        gt_vector = gt_df[lab].to_list()
        loss_sum += math.sqrt(1/len(pred_vector) * sum([(pred_vector[i]-gt_vector[i])**2 for i in range(len(pred_vector))]))
    
    return loss_sum*multiplier
    

In [37]:
mcrmse(all_res,testtt)

2.046327735099217